In [1]:
# Import Libraries
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [2]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.layers = torch.nn.Sequential(torch.nn.Linear(108, 256),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(256, 256),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(256, 128),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(128, 73))
        
    def forward(self, inputs, hidden):
        # combined = torch.tensor([inputs, hidden]).to(inputs.device)
        # print(inputs.shape)
        # print(hidden.shape)
        combined = torch.cat([inputs, hidden]).to(inputs.device)
        out = self.layers(combined)
        return out

In [4]:
model = RNN().to(device)
model.load_state_dict(torch.load('weights/rnn_4layer_256_37.pth'))

<All keys matched successfully>

In [9]:
def predict(input, model, device):
    move_mapping = ["#", "U", "U'", "U2", "D", "D'", "D2", "L", "L'", "L2", "R", 
                     "R'", "R2", "F", "F'", "F2", "B", "B'", "B2"]
    input_string = ''.join(input)
    input_int_list = [int(i) for i in input_string]
    input_tensor = torch.tensor(input_int_list)
    with torch.no_grad():
        hidden= torch.zeros(54).to(device)
        output = model(input_tensor.to(device), hidden)
        out, hidden = output[0:19], output[19::]
        pred_move = torch.argmax(out)
        prediction = move_mapping[pred_move]
    return prediction

Example of running input:
Each cube face inputed as a string.

In [10]:
predict(["302202434", "400513312", "051124551", "345533221", "005140045", "322153441"], model, device=device)

"R'"

In [11]:
predict(["000000000", "111111111", "222222222", "333333333", "444444444", "555555555"], model, device=device)

'#'

In [ ]:
# def predict(input, model, device):
#     move_mapping = ["#", "U", "U'", "U2", "D", "D'", "D2", "L", "L'", "L2", "R", 
#                      "R'", "R2", "F", "F'", "F2", "B", "B'", "B2"]
#     with torch.no_grad():
#         hidden= torch.zeros(54).to(device)
#         output = model(input.to(device), hidden)
#         out, hidden = output[0:19], output[19::]
#         pred_move = torch.argmax(out)
#         prediction = move_mapping[pred_move]
#     return prediction

In [ ]:
# input = [0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5]
# # print(predict(input ,model, device=device))